In [ ]:
from brian2 import *
import numpy as np
from time import time
set_device('cpp_standalone')
import time
import os

In [ ]:
seed(1893)

#######################
## Neuron Parameters ##
#######################

area = 300*um**2                    # surface area of the neuronal soma
Cm = (1*uF*cm**-2) * area           # membrane capacitance
El = -39.2 * mV                     # leak reversal potential
EK = -80 * mV                       # potassium reversal potential
ENa = 70 * mV                       # sodium reversal potential
g_na = (50*mS*cm**-2) * area        # maximal sodium conductance
g_kd = (5*mS*cm**-2) * area         # maximal potassium conductance
gl = (0.3*mS*cm**-2) * area         # leak conductance
VT = -30.4*mV                       # spike threshold adjustment
sigma = 4.1*mV                      # standard deviation of noisy voltage fluctuations
E_AHP = EK                          # reversal potential for AHP current

################################
## Synaptic parameters (AMPA) ##
################################

S = 16.5
E_ampa = 0*mV                       # AMPA reversal potential
g_ampa = 0.2808*nS                  # conductance increment per presynaptic spike
tau_ampa = 2*ms                     # AMPA time constant

################################
## Synaptic parameters (NMDA) ##
################################

E_nmda = 0 * mV                     # NMDA reversal potential
g_nmda = 0.0981*nS                  # conductance increment per presynaptic spike
taud_nmda = 100 * ms                # NMDA decay time constant
taur_nmda = 2 * ms                  # NMDA rise time constant
alpha_nmda = 0.5 * kHz              # NMDA activation rate

#########################
## Synaptic plasticity ##
#########################

tau_d = 800 * ms                    # recovery time constant for depression
tau_f = 50 * ms                     # recovery time constant for facilitation
x0_d = 1                            # steady-state release probability (depression)
x0_f = 0.1                          # steady-state release probability (facilitation)
fD = 0.0075                         # depression magnitude
fF = 0.00375                        # facilitation magnitude

###################
## Neuron Models ##
###################

eqs = '''
dV/dt = (noise + -gl*(V-El)-g_na*(m**3)*h*(V-ENa)-g_kd*(n**4)*(V-EK)-I_syn+I+I_AHP)/Cm : volt
dm/dt = alpha_m*(1-m)-beta_m*m : 1
dh/dt = alpha_h*(1-h)-beta_h*h : 1
dn/dt = alpha_n*(1-n)-beta_n*n : 1
dhp/dt = 0.128*exp((17.*mV-V+VT)/(18.*mV))/ms*(1.-hp)-4./(1+exp((30.*mV-V+VT)/(5.*mV)))/ms*h : 1
alpha_m = 0.32*(mV**-1)*4*mV/exprel((13*mV-V+VT)/(4*mV))/ms : Hz
beta_m = 0.28*(mV**-1)*5*mV/exprel((V-VT-40*mV)/(5*mV))/ms : Hz
alpha_h = 0.128*exp((17*mV-V+VT)/(18*mV))/ms : Hz
beta_h = 4./(1+exp((40*mV-V+VT)/(5*mV)))/ms : Hz
alpha_n = 0.032*(mV**-1)*5*mV/exprel((15*mV-V+VT)/(5*mV))/ms : Hz
beta_n = .5*exp((10*mV-V+VT)/(40*mV))/ms : Hz
noise = sigma*(2*gl*Cm)**.5*randn()/sqrt(dt) : amp (constant over dt)
I_AHP = -g_AHP*Ca*(V-E_AHP) : amp
dCa/dt = - Ca / tau_Ca : 1
I_syn = I_ampa+I_nmda : amp
I_ampa = g_ampa*(V-E_ampa)*s_ampa : amp
ds_ampa/dt = -s_ampa/tau_ampa : 1
I_nmda = g_nmda*(V-E_nmda)*s_nmda_tot/(1+exp(-0.062*V/mV)/3.57) : amp
s_nmda_tot : 1
I : amp
x : meter
y : meter
g_AHP : siemens
'''

Ne = 100                                       # number of excitatory neurons

cellsExc = NeuronGroup(Ne,
                    model=eqs,
                    threshold='V>0*mV',
                    reset='Ca += alpha_Ca',
                    refractory=2*ms,
                    method='exponential_euler')

# Pacemaker neurons
perc_pm = 9                                    # percentage of pacemaker neurons
pm_indices = np.random.choice(Ne, perc_pm, replace=False)
I_pm = np.zeros(Ne) * pA
g_AHP_pm = np.ones(Ne) * 10 * nS
I_pm[pm_indices] = 4 * pA                      # pacemaker neurons receive a small DC current
g_AHP_pm[pm_indices] = 0 * nS                  # pacemaker neurons have no AHP current

cellsExc.V = El
cellsExc.I = I_pm
cellsExc.g_AHP = g_AHP_pm
tau_Ca = 6000 * ms                             # AHP recovery time constant
alpha_Ca = 0.0002                              # spike-triggered increment for adaptation

# Spatial layout
grid_dist = 45 * umeter
cellsExc.x = '(i % int(sqrt(Ne))) * grid_dist'
cellsExc.y = '(i // int(sqrt(Ne))) * grid_dist'

#####################
## Synaptic Models ##
#####################

eqs_synE_model = '''
s_nmda_tot_post = w * S * s_nmda * x_d * x_f : 1 (summed)
ds_nmda/dt = -s_nmda/(taud_nmda)+alpha_nmda*x_nmda*(1-s_nmda) : 1 (clock-driven)
dx_nmda/dt = -x_nmda/(taur_nmda) : 1 (clock-driven)
dx_d/dt = (x0_d-x_d)/tau_d :1 (clock-driven)
dx_f/dt = (x0_f-x_f)/tau_f :1 (clock-driven)
w : 1
'''

eqs_synE_onpre = '''
s_ampa += w * S * x_d * x_f
x_nmda += 1
x_d *= (1-fD)
x_f += fF*(1-x_f)
'''

# Excitatory-to-excitatory synapses
syn_EE = Synapses(cellsExc, cellsExc, model=eqs_synE_model, on_pre=eqs_synE_onpre, method='euler')
syn_EE.connect(condition='i!=j', p=0.19)       # connection probability = 0.19

# Distributed synaptic weights (mean = 1, std = 0.7)
sd = 0.7
syn_EE.w[:] = 'clip(1.+sd*randn(), 0, 2)'

# Distance-dependent synaptic delays
Maxdelay = 25*ms
Vmax = (sqrt(2*Ne) * grid_dist) / Maxdelay
syn_EE.delay = '(sqrt((x_pre - x_post)**2 + (y_pre - y_post)**2))/Vmax'

########################
## Running Simulation ##
########################

spikesE = SpikeMonitor(cellsExc)

dur = 905*second
run(dur, report='text')

# Remove first 5 seconds of transients
sp_t = spikesE.t[spikesE.t > 5*second]-5*second
sp_i = spikesE.i[spikesE.t > 5*second]

In [ ]:
# Save spike times and indices
np.save(f'spikes_t_{perc_pm}pm.npy', sp_t/second)
np.save(f'spikes_i_{perc_pm}pm.npy', sp_i)

# Download results if running on Google Colab
from google.colab import files
files.download(f'spikes_t_{perc_pm}pm.npy')
files.download(f'spikes_i_{perc_pm}pm.npy')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#####################
## Visualization ##
#####################

import matplotlib.pyplot as plt
import numpy as np

sp_t = np.load(f'spikes_t_{perc_pm}pm.npy')
sp_i = np.load(f'spikes_i_{perc_pm}pm.npy')

fig, ax = plt.subplots(figsize=(15.92 / 2.54, 10.61 / 2.54), dpi=96)

ax.scatter(sp_t, sp_i, s=1, color='black')

ax.set_xticks([])
ax.set_yticks([])
ax.set_xlim(min(sp_t), max(sp_t))
ax.set_ylim(min(sp_i), max(sp_i))
ax.axis('off')

plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
plt.show()